In [1]:
import os
import sys
import numpy as np
import torch
import logging
import time

In [2]:
pkg_path = '/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN'
sys.path.insert(0, pkg_path)

In [3]:

from det3d import torchie
from det3d.torchie import Config
from det3d.datasets import build_dataset, build_dataloader
from det3d.models import build_detector
from det3d.torchie.apis import (
    get_root_logger,
    batch_processor
)
from det3d.torchie.trainer import get_dist_info, load_checkpoint
from det3d.torchie.trainer.utils import all_gather, synchronize
import pickle 
import time 

In [4]:
cfg_path = 'configs/waymo/voxelnet/two_stage/waymo_centerpoint_voxelnet_graphrcnn_6epoch_freeze_copy.py'

In [5]:
cfg = Config.fromfile(cfg_path)
cfg.local_rank = 0


In [6]:
logger = logging.getLogger()

In [7]:
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)


***************************model is build_from_cfg***************************
obj_type is str:  TwoStageDetector
[In build_from_cfg] building  <class 'det3d.models.detectors.two_stage.TwoStageDetector'>

***************************model is build_from_cfg***************************
obj_type is str:  VoxelNet
[In build_from_cfg] building  <class 'det3d.models.detectors.voxelnet.VoxelNet'>
Running super of VoxelNet (SingleStageDetector)

***************************model is build_from_cfg***************************
obj_type is str:  DynamicVoxelEncoder
[In build_from_cfg] building  <class 'det3d.models.readers.dynamic_voxel_encoder.DynamicVoxelEncoder'>

***************************model is build_from_cfg***************************
obj_type is str:  SpMiddleResNetFHD
[In build_from_cfg] building  <class 'det3d.models.backbones.scn.SpMiddleResNetFHD'>

***************************model is build_from_cfg***************************
obj_type is str:  RPN
[In build_from_cfg] building  <class 'de

In [8]:
device = torch.device('cpu')
map_location = 'cpu'
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    map_location = 'cuda:0'

In [9]:
ckpt = load_checkpoint(model, cfg.load_from, map_location, strict=False, logger=logger)

load_checkpoint
model loaded using LSD
model loaded with state dict


In [10]:
model = model.cuda()
model.eval()
mode = "val"

In [11]:
dataset = build_dataset(cfg.data.val)

obj_type is str:  WaymoDataset
[In build_from_cfg] building  <class 'det3d.datasets.waymo.waymo.WaymoDataset'>
Using 1 sweeps
Using 1 Frames
obj_type is str:  LoadPointCloudFromFile
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.loading.LoadPointCloudFromFile'>
obj_type is str:  LoadPointCloudAnnotations
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.loading.LoadPointCloudAnnotations'>
obj_type is str:  Preprocess
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.preprocess.Preprocess'>
obj_type is str:  AssignLabel
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.preprocess.AssignLabel'>
obj_type is str:  Reformat
[In build_from_cfg] building  <class 'det3d.datasets.pipelines.formating.Reformat'>


In [12]:
len(dataset)

1

In [13]:
from tools import calibration_waymo
import cv2
import matplotlib.pyplot as plt


In [14]:
def get_calib_object(calib_path):
    calib = calibration_waymo.get_calib_from_file(calib_path)
    calib = calibration_waymo.Calibration(calib)
    return calib
# get_calib_object(calib_path)
def get_image(image_path):
    image = cv2.imread(image_path, cv2.COLOR_BGR2RGB).astype(np.float32)
    return image
# get_image(image_path)
def get_lidar(lidar_path):
    lidar = np.fromfile(lidar_path, dtype=np.float32).reshape(-1, 5)
    lidar[:, 3] = np.tanh(lidar[:, 3])
    return lidar

In [15]:
def display_image(image):
    plt.imshow(image)
    plt.axis('off')
    plt.show()

In [16]:
def boxes3d_lidar_to_kitti_camera(boxes3d_lidar, calib):
    """
    :param boxes3d_lidar: (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center
    :param calib:
    :return:
        boxes3d_camera: (N, 7) [x, y, z, l, h, w, r] in rect camera coords
    """
    import copy
    boxes3d_lidar_copy = copy.deepcopy(boxes3d_lidar)
    xyz_lidar = boxes3d_lidar_copy[:, 0:3]
    l, w, h = boxes3d_lidar_copy[:, 3:4], boxes3d_lidar_copy[:, 4:5], boxes3d_lidar_copy[:, 5:6]
    r = boxes3d_lidar_copy[:, 6:7]

    xyz_lidar[:, 2] -= h.reshape(-1) / 2
    xyz_cam = calib.lidar_to_rect(xyz_lidar)
    # xyz_cam[:, 1] += h.reshape(-1) / 2
    r = -r - np.pi / 2
    return np.concatenate([xyz_cam, l, h, w, r], axis=-1)


In [17]:
data_path = '/media/rajeev-gupta/Drive250/SENSYN_/from_sensyn_ws_src/GraphRCNN/Ventoy/waymo_data/data/waymo/kitti/raw_data/'
segment_folders = ['segment-12974838039736660070_4586_990_4606_990_with_camera_label',
                     'segment-13145971249179441231_1640_000_1660_000_with_camera_label',
                     'segment-13182548552824592684_4160_250_4180_250_with_camera_label']

In [18]:
def draw_boxes(image, pred_boxes_cam, cam_to_img):
    import copy
    image_with_boxes = copy.deepcopy(image)
    scope_h, scope_k = image.shape[:2]
    # print('image_scope: ', scope_h, scope_k)
    for line in pred_boxes_cam[:]:
    # for line in [pred_boxes_cam[3]]:
        dims   = np.asarray([float(number) for number in line[3:6]])
        ## swap x, y, only required when reading from GD-MAE test - txt results
        tmp = dims[1]
        dims[1]=dims[0]
        dims[0]=tmp
        center = np.asarray([float(number) for number in line[0:3]])
        rot_y  = float(line[3]) + np.arctan(center[0]/center[2])
        rot_y  = float(line[3]) + float(line[6]) + np.arctan(center[0]/center[2])
        rot_y = float(line[6]) + 1.57
        # rot_y = 0
        box_3d = []
        is_bbox_inside_image_scope = True
        for i in [1,-1]:
            for j in [1,-1]:
                for k in [0,1]:
                    point = np.copy(center)
                    point[0] = center[0] + i * dims[1]/2 * np.cos(-rot_y+np.pi/2) + (j*i) * dims[2]/2 * np.cos(-rot_y)
                    point[2] = center[2] + i * dims[1]/2 * np.sin(-rot_y+np.pi/2) + (j*i) * dims[2]/2 * np.sin(-rot_y)                  
                    point[1] = center[1] - k * dims[0]
                    point = np.append(point, 1)
                    point = np.dot(cam_to_img, point)
                    point = point[:2]/point[2]
                    point = point.astype(np.int16)
                    # if point[0]>=scope_h or point[1]>=scope_k or point[0]<0 or point[1]<0:
                    if point[0]<0 or point[1]<0:
                        is_bbox_inside_image_scope = False
                    box_3d.append(point)
        if not is_bbox_inside_image_scope:
            continue
        for i in range(4):
            point_1_ = box_3d[2*i]
            point_2_ = box_3d[2*i+1]
            cv2.line(image_with_boxes, (point_1_[0], point_1_[1]), (point_2_[0], point_2_[1]), (0,255,0), 2)
        for i in range(8):
            point_1_ = box_3d[i]
            point_2_ = box_3d[(i+2)%8]
            cv2.line(image_with_boxes, (point_1_[0], point_1_[1]), (point_2_[0], point_2_[1]), (0,255,0), 2)
    return image_with_boxes

In [19]:
for seg_id, segment in enumerate(segment_folders):
    num_frames = 196
    video_out = 'waymo_GraphCE_' + str(seg_id) + '.avi'
    print(video_out)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(video_out, fourcc, 25.0, (1920, 1280))

    total_time = 0
    images_processed = []
    for index in range(num_frames):
        calib_path = os.path.join(data_path , segment, 'calib', f'{index:06d}.txt')
        image_path = os.path.join(data_path , segment, 'image/FRONT', f'{index:06d}.png')
        lidar_path = os.path.join(data_path, segment, 'velodyne', f'{index:06d}.bin')
        points = get_lidar(lidar_path)
        d0 = dataset.__getitem__(0, points)
        d0['metadata'] = {'image_prefix': '', 'num_point_features': 5, 'token': ''}
        d0['points'] = [torch.Tensor(d0['points'])]
        d0['metadata'] = [d0['metadata']]

        start_time = time.time()
        outputs = batch_processor(
                    model, d0, train_mode=False, local_rank=cfg.local_rank,
                )
        torch.cuda.synchronize()
        end_time = time.time()
        total_time += end_time-start_time
        # print('elapsed time in one batch: ', end_time-start_time)

        calib = get_calib_object(calib_path)
        cam_to_img = calib.P2

        pred_boxes = outputs[0]['box3d_lidar'].cpu().detach().numpy()
        # scores = outputs[0]['scores'].cpu().detach().numpy()
        # labels = outputs[0]['label_preds'].cpu().detach().numpy()
        try:
            image = get_image(image_path)
        except:
            print('couldn\'t load image')
        del outputs

        pred_boxes_cam = boxes3d_lidar_to_kitti_camera(pred_boxes, calib)
        image_with_boxes = draw_boxes(image, pred_boxes_cam, cam_to_img)
        # display_image(image_with_boxes / 255)
        # break
        images_processed.append(image_with_boxes)
    # break
    print(len(images_processed))
    for image_with_boxes in images_processed:
        pause_duration = 0.1
        fps = 25
        pause_frames = int(pause_duration * fps)
        for _ in range(pause_frames):
            video_writer.write(np.uint8(image_with_boxes))
    del images_processed
    # video_writer.release()
    
    print('average time: ', total_time/num_frames)
    time.sleep(4)

    

waymo_GraphCE_0.avi


/media/rajeev-gupta/Drive250/conda_envs/new_graphrcnn/lib/python3.7/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180589158/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


196
average time:  0.14837887457438878
waymo_GraphCE_1.avi
196
average time:  0.1421962064139697
waymo_GraphCE_2.avi
196
average time:  0.14630710713717401


In [20]:
del ckpt
del model

In [3]:
import threading
import time

def worker(name):
    print(f"Thread {name} starting")
    time.sleep(2)
    print(f"Thread {name} finished")

# Create and start threads
threads = []
for i in range(5):
    t = threading.Thread(target=worker, args=(i,))
    threads.append(t)
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()

print("All threads completed")


Thread 0 starting
Thread 1 starting
Thread 2 starting
Thread 3 starting
Thread 4 starting
Thread 0 finished
Thread 2 finished
Thread 4 finished
Thread 3 finished
Thread 1 finished
All threads completed


In [2]:
import multiprocessing

def worker(name):
    print(f"Process {name} starting")
    time.sleep(2)
    print(f"Process {name} finished")

# Create and start processes
processes = []
for i in range(5):
    p = multiprocessing.Process(target=worker, args=(i,))
    processes.append(p)
    p.start()

# Wait for all processes to complete
for p in processes:
    p.join()

print("All processes completed")


Process 0 starting
Process 1 starting
Process 2 starting
Process 3 starting
Process 4 starting
Process 0 finished
Process 1 finished
Process 2 finished
Process 3 finished
Process 4 finished
All processes completed


In [5]:
pc = None
if pc is not None:
    print('not')
else:
    print('None')

None
